<a href="https://colab.research.google.com/github/kushalswami20/GitGuyz/blob/main/stress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import librosa
import librosa.feature
from sklearn.preprocessing import MinMaxScaler

class AudioEmotionAnalyzer:
    def __init__(self):
        self.emotion_thresholds = {
            'happy': {'energy': 0.7, 'tempo': 120},
            'sad': {'energy': 0.3, 'tempo': 80},
            'angry': {'energy': 0.8, 'tempo': 140},
            'neutral': {'energy': 0.5, 'tempo': 100}
        }

    def normalize_value(self, value, min_val, max_val):
        """Normalize a value to range 0-1"""
        return np.clip((value - min_val) / (max_val - min_val), 0, 1)

    def extract_features(self, file_path):
        try:
            # Load the audio file
            y, sr = librosa.load(file_path, duration=30)

            features = {}

            # Normalize RMS (Root Mean Square Energy)
            rms = librosa.feature.rms(y=y)
            features['rms'] = float(np.clip(np.mean(rms) * 10, 0, 1))  # Scale and clip

            # Normalize Zero Crossing Rate
            zcr = librosa.feature.zero_crossing_rate(y)
            features['zero_crossing_rate'] = float(np.clip(np.mean(zcr), 0, 1))

            # Normalize Spectral Features
            centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
            features['spectral_centroid'] = float(self.normalize_value(np.mean(centroid), 0, sr/2))

            bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            features['spectral_bandwidth'] = float(self.normalize_value(np.mean(bandwidth), 0, sr/2))

            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            features['spectral_rolloff'] = float(self.normalize_value(np.mean(rolloff), 0, sr/2))

            # Normalize Tempo
            tempo, _ = librosa.beat.beat_track(y=y, sr=sr)
            features['tempo'] = float(self.normalize_value(tempo, 50, 200))  # Normalize between 50-200 BPM

            # Normalize MFCCs
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            for i, mfcc in enumerate(mfccs):
                features[f'mfcc_{i+1}'] = float(self.normalize_value(np.mean(mfcc), -100, 100))

            # Normalize Energy
            energy = np.sum(y**2) / len(y)
            features['energy'] = float(self.normalize_value(energy, 0, np.max(y**2)))

            return features

        except Exception as e:
            print(f"Error processing file: {str(e)}")
            return None

    def analyze_emotion(self, features):
        if features is None:
            return None

        emotion_scores = {}

        for emotion, thresholds in self.emotion_thresholds.items():
            # Calculate base scores (0-1 range)
            energy_score = 1 - abs(features['energy'] - thresholds['energy'])
            tempo_score = 1 - abs(features['tempo'] - self.normalize_value(thresholds['tempo'], 50, 200))

            # Calculate additional features (0-1 range)
            mfcc_scores = [features[f'mfcc_{i}'] for i in range(1, 14)]
            mfcc_variation = np.clip(np.std(mfcc_scores), 0, 1)
            spectral_score = features['spectral_centroid']

            # Combine scores with weights
            emotion_scores[emotion] = np.clip(
                energy_score * 0.3 +
                tempo_score * 0.3 +
                (1 - mfcc_variation) * 0.2 +
                spectral_score * 0.2,
                0, 1
            ) * 100  # Convert to percentage

        # Normalize scores to ensure they sum to 100%
        total = sum(emotion_scores.values())
        if total > 0:
            emotion_scores = {k: (v/total * 100) for k, v in emotion_scores.items()}

        stress_level = self._calculate_stress_level(features)
        dominant_emotion = max(emotion_scores.items(), key=lambda x: x[1])[0]

        return {
            'dominant_emotion': dominant_emotion,
            'emotion_scores': emotion_scores,
            'stress_level': stress_level,
            'confidence': max(emotion_scores.values())
        }

    def _calculate_stress_level(self, features):
        # Calculate stress score (0-1 range)
        stress_score = np.clip(
            features['spectral_rolloff'] * 0.4 +
            features['zero_crossing_rate'] * 0.3 +
            features['energy'] * 0.3,
            0, 1
        )

        if stress_score > 0.7:
            return "High Stress"
        elif stress_score > 0.4:
            return "Moderate Stress"
        else:
            return "Low Stress"

    def analyze_audio_file(self, file_path):
        if not os.path.exists(file_path):
            return {"error": "File not found"}

        print(f"Processing file: {file_path}")
        features = self.extract_features(file_path)

        if features is None:
            return {"error": "Error processing audio file"}

        analysis = self.analyze_emotion(features)
        return analysis

def format_results(result):
    if "error" in result:
        return f"Error: {result['error']}"

    output = "\nAnalysis Results:\n" + "="*50 + "\n"
    output += f"Dominant Emotion: {result['dominant_emotion'].upper()}\n"
    output += f"Stress Level: {result['stress_level']}\n"
    output += f"Confidence Score: {result['confidence']:.1f}%\n\n"

    output += "Detailed Emotion Scores:\n" + "-"*30 + "\n"
    for emotion, score in result['emotion_scores'].items():
        bars = "█" * int(score/5)  # Visual representation of score
        output += f"{emotion.capitalize():8}: {score:5.1f}% |{bars}\n"

    return output

# Example usage
if __name__ == "__main__":
    analyzer = AudioEmotionAnalyzer()
    file_path = "/content/drive/MyDrive/Music/Color_Out_-_Host.wav"

    print("\nAnalyzing audio file...")
    result = analyzer.analyze_audio_file(file_path)
    print(format_results(result))


Analyzing audio file...
Processing file: /content/drive/MyDrive/Music/Color_Out_-_Host.wav

Analysis Results:
Dominant Emotion: SAD
Stress Level: Low Stress
Confidence Score: 26.4%

Detailed Emotion Scores:
------------------------------
Happy   :  24.7% |████
Sad     :  26.4% |█████
Angry   :  23.3% |████
Neutral :  25.6% |█████



<ipython-input-1-e10defda728d>:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features['tempo'] = float(self.normalize_value(tempo, 50, 200))  # Normalize between 50-200 BPM
